# Find the seed of an existing image

### Imports

In [ ]:
import sys
sys.path.append('../')

import os
import torch
from torch import nn
import torchvision as tv
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from training.generator import Generator
from training import utils
from training.settings import *

utils.reset_rand()

### Check GPU

In [ ]:
utils.check_gpu()

### Import the model

In [ ]:
MODEL_PATH = 'models/faces_256.pt'

model = Generator().to(DEVICE)
model.load_state_dict(torch.load(os.path.join('..', MODEL_PATH), map_location = DEVICE))
model.eval()
model.requires_grad_(False)

### Mean W image

In [ ]:
image = model.generate_one(0.0)

plt.imshow(image)
plt.axis('off')
plt.show()

### Settings

In [ ]:
IMAGE_PATH = 'test.png'		# Path to the image to project
SAVE_DIR = 'projector'		# Directory to save the images to
NB_STEPS = 12_000			# Number of steps to use for projection
MAX_LEARNING_RATE = 0.1		# Maximum learning rate to use
WARMUP_STEPS = 500			# Number of steps to use for warmup
COOLDOWN_STEPS = 500		# Number of steps to use for cooldown
W_NOISE_SCALE = 0.1			# The scale of the noise on W
W_NOISE_STEPS = 7_000		# Number of steps using noise on W
NOISE_REG_STRENGTH = 100	# Strength of the noise regularization
NB_SAVED_IMAGES = 1_000		# Number of images to save

### Import the image and VGG16

In [ ]:
convert = transforms.Compose([
	transforms.Resize(IMAGE_SIZE, interpolation = transforms.InterpolationMode.LANCZOS),
	transforms.CenterCrop(IMAGE_SIZE),
	transforms.ToTensor()
])

file = os.path.join('..', IMAGE_PATH)

if NB_CHANNELS == 1:
	target = Image.open(file).convert('L')
elif NB_CHANNELS <= 3:
	target = Image.open(file).convert('RGB')
else:
	target = Image.open(file).convert('RGBA')

target = convert(target) * 2.0 - 1.0

if NB_CHANNELS == 2:
	target = target[:2]

target = target.to(DEVICE).unsqueeze(0).detach().requires_grad_(False)

### Import VGG16

In [ ]:
vgg16_model = tv.models.vgg16(weights = tv.models.VGG16_Weights.DEFAULT).to(DEVICE)
vgg16_model = nn.Sequential(*list(vgg16_model.features.children()))
vgg16_model.eval()
vgg16_model.requires_grad_(False)

vgg16_transform = tv.models.VGG16_Weights.DEFAULT.transforms()


def vgg16(x: torch.Tensor) -> torch.Tensor:

	x = (x + 1.0) / 2.0
	x = vgg16_transform(x)
	x = vgg16_model(x)

	return x.requires_grad_(True)


target_features = vgg16(target)

### Initialize the seed

In [ ]:
ws = model.gen_w(MEAN_W_SAMPLES)
w_mean = ws.mean(0, keepdims = True).detach().requires_grad_(False)
w_std = ((ws - w_mean).square().sum() / MEAN_W_SAMPLES).sqrt().item()

w = w_mean.clone().requires_grad_(True)
noise = model.gen_noise(1)
noise = [n.requires_grad_(True) for n in noise]

optimizer = torch.optim.Adam([w] + noise, lr = 0.0)

### Training

In [ ]:
for step in range(NB_STEPS):

	optimizer.zero_grad(set_to_none = True)

	lr_start = min(1.0, step / WARMUP_STEPS)
	lr_end = min(1.0, (NB_STEPS - step) / COOLDOWN_STEPS)
	lr_end = 0.5 - 0.5 * np.cos(lr_end * np.pi)
	lr = LEARNING_RATE * lr_start * lr_end

	for param_group in optimizer.param_groups:
		param_group['lr'] = lr

	w_noise_scale = w_std * W_NOISE_SCALE * max(0.0, 1.0 - step / W_NOISE_STEPS) ** 2
	w_noise = w_noise_scale * torch.randn_like(w)

	gen_image = model.synthesis(w + w_noise, noise)
	gen_features = vgg16(gen_image)

	pixel_loss = (gen_image - target).square().mean()
	gen_temp = nn.functional.interpolate(gen_image, scale_factor = 0.5, mode = 'bilinear')
	target_temp = nn.functional.interpolate(target, scale_factor = 0.5, mode = 'bilinear')

	while True:

		pixel_loss = pixel_loss + (gen_temp - target_temp).square().mean()
		gen_temp = nn.functional.interpolate(gen_temp, scale_factor = 0.5, mode = 'bilinear')
		target_temp = nn.functional.interpolate(target_temp, scale_factor = 0.5, mode = 'bilinear')

		if gen_temp.shape[2] == 1:
			break

	features_loss = (gen_features - target_features).square().mean()
	noise_reg = 0.0

	for n in noise:
		while True:

			noise_reg = noise_reg + (n * torch.roll(n, shifts = 1, dims = 2)).mean().square()
			noise_reg = noise_reg + (n * torch.roll(n, shifts = 1, dims = 3)).mean().square()
			n = nn.functional.avg_pool2d(n, kernel_size = 2)

			if n.shape[2] < 8:
				break

	loss = pixel_loss + math.log2(IMAGE_SIZE) * features_loss + NOISE_REG_STRENGTH * noise_reg

	loss.backward()
	optimizer.step()

	with torch.no_grad():

		for n in noise:
			n.sub(n.mean())
			n.mul(n.square().mean().rsqrt())

		if not os.path.exists(os.path.join('..', SAVE_DIR)):
			os.makedirs(os.path.join('..', SAVE_DIR))

		if step % (NB_STEPS // NB_SAVED_IMAGES) == 0:
			save_image = utils.denormalize(model.synthesis(w, noise).squeeze(0))
			Image.fromarray(save_image).save(os.path.join('..', SAVE_DIR, f'{step // (NB_STEPS // NB_SAVED_IMAGES)}.png'))
			Image.fromarray(save_image).save(os.path.join('..', 'projector.png'))

		print(f'Steps: {step:,} / {NB_STEPS}  ||  Pixel Loss: {pixel_loss.item():.4f} | Features Loss: {math.log2(IMAGE_SIZE) * features_loss.item():.4f} | ' + \
			f'Noise Regularisation: {NOISE_REG_STRENGTH * noise_reg.item():.4f}          ', end = '\r')